# SEIHRD model class experiments

Utilities for train, forecasting and plotting are now provided in nb_utils.
Documentation for the helper functions are available as docstrings.

Set the appropriate training parameters and forecast parameters in the 
corresponding cells.

train_eval   
> Train and evaluate models (currently we only support SEIR family). 
  Default configuration parameters for train and test files are 
  provided as train_config.json and test_config.json respectively.
  Search paramters, loss functions etc, are specified in the config
  files.

forecast, plot
> Generate forecasts using a trained model and plotting utilities.

In [ ]:
import json
import pandas as pd
import mlflow
import os

from datetime import datetime, timedelta

from nb_utils import plot
from nb_utils import plot_m1, plot_m2, plot_m3
from nb_utils import train_eval_plot

## Load default configurations

In [ ]:
with open('train_config.json') as f_train, \
    open('test_config.json') as f_test, \
    open('forecast_config.json') as f_forecast, \
    open('mlflow_credentials.json') as f_cred:
    default_train_config = json.load(f_train)
    default_test_config = json.load(f_test)
    default_forecast_config = json.load(f_forecast)
    mlflow_credentials = json.load(f_cred)

## Set the model class

In [ ]:
model_class = 'SEIHRD'
default_train_config['model_class'] = model_class
default_test_config['model_class'] = model_class
default_forecast_config['model_class'] = model_class

## MLflow setup

Set MLflow username, password and experiment.

In [ ]:
os.environ['MLFLOW_TRACKING_USERNAME'] = mlflow_credentials['username']
os.environ['MLFLOW_TRACKING_PASSWORD'] = mlflow_credentials['password']

mlflow.set_tracking_uri("http://ec2-54-175-207-176.compute-1.amazonaws.com")
mlflow.set_experiment("seihrd_experiments")

## Set experiments

In [ ]:
experiments = [
    {'region':['pune'],'region_type':'district'}
]

current_day = datetime.now().date() - timedelta(4) # Forecast starts from the day after current day
forecast_length = 30 # Length of forecast interval (days)

## Run experiments

In [ ]:
# Official data, combined

for experiment in experiments:
    train_eval_plot(experiment['region'], experiment['region_type'], 
                    current_day, forecast_length,
                    default_train_config, default_test_config, default_forecast_config,
                    max_evals = 1000, data_source = 'tracker_district_daily', 
                    mlflow_log = False, mlflow_run_name = "Testing combined data")